In [4]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd

In [2]:
from transformers import pipeline

# Load the pretrained sentiment-analysis pipeline
# By default, the pipeline uses a variant of BERT, called distilbert-base-uncased-finetuned-sst-2-english.
sentiment_analyzer = pipeline("sentiment-analysis")

/home/bigburrito/Deep/deepEnvironment/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/home/bigburrito/Deep/deepEnvironment/lib/python3.10/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance,

In [32]:
results = sentiment_analyzer('im sad . im happy . ')
results

[{'label': 'POSITIVE', 'score': 0.9997468590736389}]

In [61]:
class TextCaptionDataset(Dataset):

    def __init__(self, text_ner_path, caption_path):
        self.tweets = pd.read_csv(text_ner_path, sep=';')
        self.captions = pd.read_csv(caption_path, sep=';')
        self.data = self.tweets.merge(self.captions, left_on='id', right_on='image_name', how='inner')
    
    def __len__(self):
        return len(self.captions)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        return item['id'], item['tweet'], item['caption'], item['ner']

In [62]:
data = TextCaptionDataset('../data/csv/text_and_ner_train.csv', '../data/csv/captionsImages.csv')

In [58]:
data[0][3]

' O  O  O  O  O  O  O  O  B-PER  I-PER  O  O  O  O  O  O  O  O  O  O  O  O'

In [ ]:
df = pd.DataFrame(columns=['id', 'tweet', 'caption', 'ner', 'sa'])
k = 0
for id, tweet, caption, ner in data:
    ner += " O" * len(caption.split())
    result = sentiment_analyzer(tweet + caption)
    df.loc[len(df)] = [id, tweet, caption, ner, result[0]['label']]


df.to_csv('../data/csv/train.csv', sep=';')

ValueError: cannot set a row with mismatched columns